In [1]:
from dotenv import load_dotenv
import os
import requests
import numpy as np
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data4
import pandas as pd
import random
import credentials as cred

load_dotenv()

True

In [4]:
spotify_token  = cred.spotifyToken()
headers = {"Authorization":f"Bearer {spotify_token}"}

In [5]:
url_base = "https://api.spotify.com/v1/playlists/"
playlist_link = 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M?si=b705194596334305'

In [6]:
test_query = cred.playlist_query ()

In [7]:
response = cred.playlist_response()

In [8]:
track_items_dict = cred.track_items ()

In [9]:
track_items_dict = cred.track_audio_features ()

In [10]:
track_items_dict = cred.get_lyrics ()

Searching for "Die For You - Remix" by The Weeknd...
Done.
Searching for "Flowers" by Miley Cyrus...
Done.
Searching for "Boy's a liar Pt. 2" by PinkPantheress...
Done.
Searching for "Kill Bill" by SZA...
Done.
Searching for "Escapism." by RAYE...
Done.
Searching for "Creepin' (with The Weeknd & 21 Savage)" by Metro Boomin...
Done.
Searching for "Calm Down (with Selena Gomez)" by Rema...
Done.
Searching for "TQG" by KAROL G...
Done.
Searching for "As It Was" by Harry Styles...
Done.
Searching for "Players" by Coi Leray...
Done.
Searching for "CUFF IT" by Beyoncé...
Done.
Searching for "Anti-Hero" by Taylor Swift...
Done.
Searching for "golden hour" by JVKE...
Done.
Searching for "Shakira: Bzrp Music Sessions, Vol. 53" by Bizarrap...
Done.
Searching for "Unholy (feat. Kim Petras)" by Sam Smith...
Done.
Searching for "Just Wanna Rock" by Lil Uzi Vert...
Done.
Searching for "Love Again" by The Kid LAROI...
Done.
Searching for "Heaven" by Niall Horan...
Done.
Searching for "LET GO" by Cent

In [11]:
track_items_dict = cred.track_sentiment ()

In [12]:
df = pd.DataFrame(track_items_dict)

In [13]:
df.head()

,Name,Artist,Popularity,Song ID,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Lyrics,Positive,Neutral,Negative,Compound
0,Die For You - Remix,The Weeknd,78,7oDd86yk8itslrA9HRP2ki,0.531,0.525,1,-6.500,0,0.0671,0.2320,0.000000,0.4410,0.502,66.900,TranslationsEspañolItalianoDeutschDie For You ...,0.125,0.625,0.250,-0.9982
1,Flowers,Miley Cyrus,100,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,TranslationsEspañolPortuguêsKiswahiliDeutschIt...,0.517,0.439,0.044,0.9998
2,Boy's a liar Pt. 2,PinkPantheress,91,6AQbmUe0Qwf5PZnt4HmTXv,0.696,0.809,5,-8.254,1,0.0500,0.2520,0.000128,0.2480,0.857,132.962,TranslationsPortuguêsTürkçeBoy’s a liar Pt. 2 ...,0.195,0.681,0.124,0.9664
3,Kill Bill,SZA,92,1Qrg8KqiBpW07V7PNxwwwL,0.644,0.735,8,-5.747,1,0.0391,0.0521,0.144000,0.1610,0.418,88.980,TranslationsEspañolPortuguêsItalianoTürkçeDeut...,0.203,0.609,0.188,0.6321
4,Escapism.,RAYE,89,5Z2MiIZ5I3jJvvmeWMLbOQ,0.538,0.742,2,-5.355,1,0.1140,0.1380,0.000047,0.0934,0.250,96.107,TranslationsTürkçeEspañolPortuguêsEscapism. Ly...,0.083,0.823,0.095,-0.7889


In [17]:
import nltk
import spacy
import gensim

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Download spaCy resources
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
# Load the song lyrics
#with open('lyrics.txt', 'r') as f:
#    lyrics = f.read()
lyrics = track_items_dict['Lyrics'][0]

# Tokenize the lyrics
tokens = nltk.word_tokenize(lyrics.lower())

# Remove stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))
tokens = [token for token in tokens if not token in stop_words]

# Remove punctuation
tokens = [token for token in tokens if token.isalpha()]


In [22]:
# Create a dictionary from the tokenized lyrics
dictionary = gensim.corpora.Dictionary([tokens])

# Convert the tokenized lyrics into a bag-of-words representation
bow_corpus = [dictionary.doc2bow(tokens)]

# Train an LDA model on the bag-of-words representation
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=1,
    random_state=42
)

# Extract the main topic from the LDA model
# Extract the main topic from the LDA model
topic = lda_model.print_topics()[0][1]

# Split the topic into individual words with proportions
topic_words = topic.split('+')

# Extract the word with the highest proportion
representative_word = max(topic_words, key=lambda x: float(x.split('*')[0])) \
                      .split('*')[1].strip('" ')

# Print the representative word
print('The theme of the song is:', representative_word)


The theme of the song is: would


In [28]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [30]:
# Calculate the average vector for all the words in the topic
topic_words = topic.split("+")
topic_vector = np.zeros((nlp.vocab.vectors.shape[1],), dtype="float32")
for word in topic_words:
    word = word.split("*")[1].strip().strip('"')
    word_vector = nlp.vocab[word].vector
    topic_vector += word_vector
topic_vector /= len(topic_words)

# Find the word closest to the topic vector in the Word Embedding space
representative_word = nlp.vocab.vectors.most_similar(
    topic_vector.reshape(1, -1), n=1
)[0][0]

# Print the representative word
print('The theme of the song is:', representative_word)


ValueError: [E010] Word vectors set to length 0. This may be because you don't have a model installed or loaded, or because your model doesn't include word vectors. For more info, see the docs:
https://spacy.io/usage/models

In [35]:
pd.read_fwf('../data/text.txt')

,[,1.,0.,0..1,0..2,0..3,0..4,0.],"During the period of falling in love, each time that we met and especially when we had not met for a long time.",Unnamed: 9,Unnamed: 10,Unnamed: 11
0,[,0.0,1.0,0.0,0.0,0.0,0.0,0.],When I was involved in a traffic accident.,NaN,NaN,NaN
1,[,0.0,0.0,1.0,0.0,0.0,0.0,0.],When I was driving home after several days of...,NaN,NaN,NaN
2,[,0.0,0.0,0.0,1.0,0.0,0.0,0.],When I lost the person who meant the most to me.,NaN,NaN,NaN
3,[,0.0,0.0,0.0,0.0,1.0,0.0,0.],The time I knocked a deer down - the sight of ...,NaN,NaN,NaN
4,[,0.0,0.0,0.0,0.0,0.0,1.0,0.],When I did not speak the truth.,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7474,[,0.0,0.0,1.0,0.0,0.0,0.0,0.],Two years back someone invited me to be the tu...,seco,d,ear
7475,[,0.0,0.0,0.0,1.0,0.0,0.0,0.],I had taken the responsibility to do something...,ent a,d,elt
7476,[,0.0,0.0,0.0,0.0,1.0,0.0,0.],I was at home and I heard a loud sound of spit...,NaN,NaN,NaN
7477,[,0.0,0.0,0.0,0.0,0.0,1.0,0.],I did not do the homework that the teacher had...,NaN,NaN,NaN


In [78]:
spotify_genres = pd.read_excel('../data/spotify_genre_final.xlsx')

In [82]:
spotify_genres['Genre'].value_counts()

pop       1161
rock      1136
latin     1036
hiphop     971
edm        956
rap        926
r&b        731
Name: Genre, dtype: int64

In [83]:
spotify_genres.columns

Index(['Genre', 'Title', 'Album_cover_link', 'Artist', 'duration_ms',
       'explicit', 'id', 'popularity', 'release_date',
       'release_date_precision', 'total_tracks', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [87]:
spotify_genres = spotify_genres[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo']]

In [89]:
spotify_genres.head()

,Genre,Title,Artist,popularity,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,rock,Baba O'Riley,The Who,75,3qiyyUfYe7CRYLucrPmulD,0.489,0.724,5,-8.367,1,0.0352,0.31300,0.185000,0.2870,0.150,117.292
1,rock,More Than a Feeling,Boston,78,1QEEqeFIZktqIpPI4jSVSF,0.377,0.681,7,-8.039,1,0.0298,0.00088,0.002300,0.0504,0.285,108.789
2,rock,(Don't Fear) The Reaper,Blue Öyster Cult,76,5QTxFnGygVM4jFQiBovmRo,0.333,0.927,9,-8.550,0,0.0733,0.00290,0.000208,0.2970,0.385,141.466
3,rock,Jump - 2015 Remaster,Van Halen,78,7N3PAbqfTjSEU1edb2tY8j,0.572,0.835,0,-6.219,1,0.0317,0.17100,0.000377,0.0702,0.795,129.981
4,rock,Stairway to Heaven - Remaster,Led Zeppelin,79,5CQ30WqJwcep0pYcV4AMNc,0.338,0.340,9,-12.049,0,0.0339,0.58000,0.003200,0.1160,0.197,82.433


In [99]:
from sklearn.model_selection import train_test_split

In [100]:
# data to be used in the model will be 
data = spotify_genres # pd.read_excel('../data/spotify_genre_final.xlsx')[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [101]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

In [102]:
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [103]:
train_features = train_data[features]
train_target = train_data['Genre']

In [104]:
test_features = test_data[features]
test_target = test_data['Genre']

In [118]:
rfc.fit(train_features, train_target)

RandomForestClassifier()

In [106]:
predictions = rfc.predict(test_features)

In [107]:
accuracy = (predictions == test_target).sum() / len(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4735067437379576


In [108]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'max_features': ['sqrt', 'log2']
}

# Initialize a random forest classifier
rfc = RandomForestClassifier()

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(train_features, train_target)

# Print the best hyperparameters and the corresponding accuracy
print(f"Best parameters: {grid_search.best_params_}")
print(f"Accuracy: {grid_search.best_score_}")


Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}
Accuracy: 0.48729413470477356


In [109]:
import joblib

# Save the trained model
joblib.dump(rfc, 'genre_mode_spotify_project')

['genre_mode_spotify_project']

In [127]:
rfc = RandomForestClassifier(n_estimators = 100, max_depth = 5, max_features = 'log2')
rfc.fit(train_features, train_target)
joblib.dump(rfc, 'new_genre_mode_spotify_project')

['new_genre_mode_spotify_project']

In [128]:
# Load the saved model
rfc = joblib.load('new_genre_mode_spotify_project')

# Prepare the new data

new_data = pd.DataFrame({
    'danceability': [0.5],
    'energy': [0.5],
    'key': [1],
    'loudness': [-6.5],
    'mode': [0],
    'speechiness': [0.06],
    'acousticness': [0.23],
    'instrumentalness': [0.0],
    'liveness': [0.44],
    'valence': [0.5],
    'tempo': [66.9]
})

# Make predictions on the new data
predictions = rfc.predict(new_data)
print(predictions)


['pop']


In [111]:
track_items_dict.keys()

dict_keys(['Name', 'Artist', 'Popularity', 'Song ID', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Lyrics', 'Positive', 'Neutral', 'Negative', 'Compound'])

In [116]:
pd.DataFrame({
    'danceability': [0.5],
    'energy': [0.5],
    'key': [1],
    'loudness': [-6.5],
    'mode': [0],
    'speechiness': [0.06],
    'acousticness': [0.23],
    'instrumentalness': [0.0],
    'liveness': [0.44],
    'valence': [0.5],
    'tempo': [66.9]
})

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.5,0.5,1,-6.5,0,0.06,0.23,0.0,0.44,0.5,66.9


In [115]:
df.iloc[0]

Name                                              Die For You - Remix
Artist                                                     The Weeknd
Popularity                                                         78
Song ID                                        7oDd86yk8itslrA9HRP2ki
Danceability                                                    0.531
Energy                                                          0.525
Key                                                                 1
Loudness                                                         -6.5
Mode                                                                0
Speechiness                                                    0.0671
Acousticness                                                    0.232
Instrumentalness                                                  0.0
Liveness                                                        0.441
Valence                                                         0.502
Tempo               

In [84]:
df.columns

Index(['Name', 'Artist', 'Popularity', 'Song ID', 'Danceability', 'Energy',
       'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness',
       'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Lyrics',
       'Positive', 'Neutral', 'Negative', 'Compound'],
      dtype='object')

PDF Panda R., Malheiro R. & Paiva R. P. (2018). “Novel audio features for music emotion recognition”. IEEE Transactions on Affective Computing (IEEE early access). DOI: 10.1109/TAFFC.2018.2820691.

PDFPanda R., Malheiro R., Paiva R. P. (2018). “Musical Texture and Expressivity Features for Music Emotion Recognition”. 19th International Society for Music Information Retrieval Conference – ISMIR 2018, Paris, France.

In [15]:
emotions_list = ['amusement', 'joy', 'eroticism', 'beauty', 'relaxation', 'sadness', 'dreaminess', 'triumph', 'anxiety', 'scariness', 'annoyance', 'defiance', 'feeling pumped up']
emotions_list

['amusement',
 'joy',
 'eroticism',
 'beauty',
 'relaxation',
 'sadness',
 'dreaminess',
 'triumph',
 'anxiety',
 'scariness',
 'annoyance',
 'defiance',
 'feeling pumped up']

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

# Load stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load Porter Stemmer
stemmer = PorterStemmer()

# Load song lyrics
lyrics = df.iloc[1]['Lyrics']

# Remove punctuation and convert to lowercase
lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
lyrics = lyrics.lower()

# Tokenize words
words = word_tokenize(lyrics)

# Remove stop words and stem words
words = [stemmer.stem(word) for word in words if word not in stop_words]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
audio_features_request_base = "https://api.spotify.com/v1/audio-features/"
test_request_here = audio_features_request_base + '2HvvnVkD1kcnaYjgKO5E5c'
audio_features_here = requests.get(test_request_here, headers=headers).json() 

In [27]:
def one_song_feature (song_id):
    headers = cred.query_header (spotify_token)
    audio_features_request_base = "https://api.spotify.com/v1/audio-features/"
    song_request = audio_features_request_base + song_id
    features = requests.get(song_request, headers=headers).json()
    return features

In [30]:
test_features = one_song_feature ('0yLdNVWF3Srea0uzk55zFn')

In [49]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Create Bag-of-Words matrix
vectorizer = CountVectorizer()
X_lyrics = vectorizer.fit_transform([lyrics]).toarray()

In [56]:
X_lyrics
counts = np.sum(X_lyrics, axis=0)
top_indices = np.argsort(counts)[::-1][:9]

top_indices
word_counts = dict(zip(vocabulary, counts))


array([ 9, 39, 36,  4, 79,  2, 43, 78, 42])

In [61]:
test_array = np.array([test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])
test_array

array([ 7.07000e-01,  6.81000e-01, -4.32500e+00,  6.68000e-02,
        6.32000e-02,  5.15000e-06,  3.22000e-02,  6.46000e-01,
        1.17999e+02])

In [41]:
print(X_lyrics.shape)
print(np.array([[test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]]).shape)

(1, 80)
(1, 9)


In [39]:

# Create feature vector
X = np.concatenate((X_lyrics, np.array([[test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]])))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 80 and the array at index 1 has size 9

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define emotions list
emotions_list = ['amusement', 'joy', 'eroticism', 'beauty', 'relaxation', 'sadness', 'dreaminess', 'triumph', 'anxiety', 'scariness', 'annoyance', 'defiance', 'feeling pumped up']

# Load pre-trained random forest classifier
clf = joblib.load('random_forest_classifier.joblib')

# Predict emotion
y_pred = clf.predict([X])[0]

# Print predicted emotion
print(emotions_list[y_pred])

